## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Arraial Do Cabo,-22.9661,-42.0278,scattered clouds,77.00,73,40,28.77,BR,2021-04-20 13:14:27
1,1,Ordzhonikidze,52.4411,61.7408,clear sky,42.19,30,0,9.22,KZ,2021-04-20 13:14:27
2,2,Kapaa,22.0752,-159.3190,clear sky,69.80,94,1,5.75,US,2021-04-20 13:14:28
3,3,Punta Arenas,-53.1500,-70.9167,few clouds,42.80,75,20,13.80,CL,2021-04-20 13:14:28
4,4,Zhanaozen,43.3000,52.8000,overcast clouds,80.56,21,100,10.76,KZ,2021-04-20 13:14:28


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Arraial Do Cabo,-22.9661,-42.0278,scattered clouds,77.00,73,40,28.77,BR,2021-04-20 13:14:27
4,4,Zhanaozen,43.3000,52.8000,overcast clouds,80.56,21,100,10.76,KZ,2021-04-20 13:14:28
6,6,San Cristobal,7.7669,-72.2250,few clouds,87.80,33,20,23.02,VE,2021-04-20 13:14:29
9,9,Ciudad Bolivar,8.1222,-63.5497,few clouds,82.22,56,17,13.47,VE,2021-04-20 13:14:29
11,11,Saint-Pierre,-21.3393,55.4781,heavy intensity rain,75.20,94,75,11.50,RE,2021-04-20 13:14:30
12,12,Rikitea,-23.1203,-134.9692,few clouds,77.47,67,21,7.16,PF,2021-04-20 13:14:30
18,18,Lorengau,-2.0226,147.2712,overcast clouds,81.27,78,100,8.46,PG,2021-04-20 13:14:32
20,20,Matara,5.9485,80.5353,scattered clouds,81.63,78,37,10.38,LK,2021-04-20 13:14:32
27,27,Black River,18.0264,-77.8487,few clouds,79.00,69,20,4.61,JM,2021-04-20 13:14:34
28,28,Inhambane,-23.8650,35.3833,broken clouds,77.00,73,75,8.05,MZ,2021-04-20 13:14:35


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Lat                    218
Lng                    218
Current Description    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Country                214
Date                   218
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,77.00,scattered clouds,-22.9661,-42.0278,
4,Zhanaozen,KZ,80.56,overcast clouds,43.3000,52.8000,
6,San Cristobal,VE,87.80,few clouds,7.7669,-72.2250,
9,Ciudad Bolivar,VE,82.22,few clouds,8.1222,-63.5497,
11,Saint-Pierre,RE,75.20,heavy intensity rain,-21.3393,55.4781,
12,Rikitea,PF,77.47,few clouds,-23.1203,-134.9692,
18,Lorengau,PG,81.27,overcast clouds,-2.0226,147.2712,
20,Matara,LK,81.63,scattered clouds,5.9485,80.5353,
27,Black River,JM,79.00,few clouds,18.0264,-77.8487,
28,Inhambane,MZ,77.00,broken clouds,-23.8650,35.3833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,77.00,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
4,Zhanaozen,KZ,80.56,overcast clouds,43.3000,52.8000,K.korabay 28A
6,San Cristobal,VE,87.80,few clouds,7.7669,-72.2250,Posada Villaven C.A.
9,Ciudad Bolivar,VE,82.22,few clouds,8.1222,-63.5497,Hotel Edi
11,Saint-Pierre,RE,75.20,heavy intensity rain,-21.3393,55.4781,Lindsey Hôtel
...,...,...,...,...,...,...,...
697,Mizan Teferi,ET,81.70,overcast clouds,6.9833,35.5833,Hotel Salayish | ሆቴል ሳላይሽ
699,Lubao,PH,82.40,broken clouds,14.9167,120.5500,Hacienda Gracia
700,Olinda,BR,80.60,broken clouds,-8.0089,-34.8553,Hotel Albergue de Olinda
702,Khash,IR,79.48,clear sky,28.2211,61.2158,دفتر پیشخوان دولت جمال زهی (خالد)


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
clean_hotel_Lat_Lng_df = clean_hotel_df[["Lat", "Lng", "Max Temp", "Current Description"]]


In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_Lat_Lng_df[["Lat", "Lng"]] 
Max_temp = clean_hotel_Lat_Lng_df["Max Temp"] 
Current_conditions = clean_hotel_Lat_Lng_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


#11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))